In [11]:
import io
import networkx as nx
import numpy as np
import pandas as pd
import requests
from sklearn import cluster, metrics
import gzip
import warnings
warnings.simplefilter("ignore")

import graph2vec
from graph2vec.graph import _sparse_normalize_rows

In [2]:
def evalClusteringOnLabels(clusters, groupLabels, verbose=True):
    results = []
    results.append(metrics.adjusted_mutual_info_score(clusters, groupLabels))
    results.append(metrics.adjusted_rand_score(clusters, groupLabels))
    results.append(metrics.fowlkes_mallows_score(clusters, groupLabels))
    if verbose:
        print("adj. MI score:   {0:.2f}".format(results[0]))
        print("adj. RAND score: {0:.2f}".format(results[1]))
        print("F-M score:       {0:.2f}".format(results[2]))
    return np.array(results)

In [3]:
# Data from http://snap.stanford.edu/data/email-Eu-core.html
# Edge list
res = requests.get('http://snap.stanford.edu/data/email-Eu-core.txt.gz', verify=False)
edges = gzip.GzipFile(fileobj=io.BytesIO(res.content))
edges = pd.read_csv(io.StringIO(edges.read().decode()), header=None, sep=' ')
edges.columns = ['src', 'dest']
# cluster labels per node
res = requests.get('http://snap.stanford.edu/data/email-Eu-core-department-labels.txt.gz', verify=False)
labels = gzip.GzipFile(fileobj=io.BytesIO(res.content))
labels = pd.read_csv(io.StringIO(labels.read().decode()), header=None, sep=' ')
labels.columns = ['node', 'cluster']

In [4]:
G = nx.Graph()
G.add_edges_from([(t.src, t.dest) for t in edges.itertuples()])

# Graph nodes should be ordered ints
if not (np.array(list(G.nodes)) == np.arange(len(list(G.nodes)))).all():
    raise ValueError("graph nodes are not ordered")

In [12]:
embed_size = 32
n_clusters = labels.cluster.nunique()

n2v = graph2vec.Node2Vec(
    walklen=10,
    epochs=100,
    return_weight=1.0,
    neighbor_weight=1.0,
    threads=8,
    w2vparams={'window': 4,
               'size': embed_size, 
               'negative': 3, 
               'iter': 10,
               'ns_exponent': 0.5,
               'batch_words': 128}
)
n2v.fit(G, verbose=True)

print("retrieving embeddings")
nodes = list(G.nodes)
embeds = np.empty((len(nodes), embed_size))
for i in nodes:
    embeds[i] = n2v.predict(i)

print("Clustering")
# Density based methods don't seem to perform well
# Since we know ahead of time the nb of clusters
# Agglomerative is best performing and has stable results
agglo = cluster.AgglomerativeClustering(
    n_clusters=n_clusters, 
    affinity='cosine', # euclidean with Ward is slightly worse
    linkage='average'
).fit(embeds).labels_

x = evalClusteringOnLabels(agglo, labels.cluster)

Making walks... Done, T=0.10
Mapping Walk Names... Done, T=0.59
Training W2V... Done, T=12.17
retrieving embeddings
Clustering
adj. MI score:   0.65
adj. RAND score: 0.54
F-M score:       0.56


In [13]:
import umap

ump = umap.UMAP(
    n_neighbors=15,
    min_dist=0.01,
    metric='euclidean',
    n_components=embed_size,
)
umpembed = ump.fit_transform(
    _sparse_normalize_rows(nx.adjacency_matrix(G))
)
umpagglo = cluster.AgglomerativeClustering(
    n_clusters=n_clusters, 
    affinity='cosine', 
    linkage='average'
).fit(umpembed).labels_

print("UMAP Score:")
x = evalClusteringOnLabels(umpagglo, labels.cluster)

UMAP Score:
adj. MI score:   0.65
adj. RAND score: 0.56
F-M score:       0.58


In [14]:
from sklearn import decomposition

pcae = decomposition.PCA(
    n_components=embed_size,
    whiten=True
)
pcae = pcae.fit_transform(
    _sparse_normalize_rows(nx.adjacency_matrix(G)).todense()
)
pcae = cluster.AgglomerativeClustering(
    n_clusters=n_clusters, 
    affinity='cosine', 
    linkage='average'
).fit(pcae).labels_

print("PCA Score:")
x = evalClusteringOnLabels(pcae, labels.cluster)

PCA Score:
adj. MI score:   0.52
adj. RAND score: 0.41
F-M score:       0.44


In [15]:
from sklearn import manifold

pcae = manifold.SpectralEmbedding(
    n_components=embed_size,
    affinity='nearest_neighbors',
)
pcae = pcae.fit_transform(
    _sparse_normalize_rows(nx.adjacency_matrix(G)).todense()
)
pcae = cluster.AgglomerativeClustering(
    n_clusters=n_clusters, 
    affinity='cosine', 
    linkage='average'
).fit(pcae).labels_

print("Spectral Score:")
x = evalClusteringOnLabels(pcae, labels.cluster)

Spectral Score:
adj. MI score:   0.53
adj. RAND score: 0.40
F-M score:       0.43


In [16]:
pcae = manifold.Isomap(
    n_components=embed_size,
    n_neighbors=10,
)
pcae = pcae.fit_transform(
    _sparse_normalize_rows(nx.adjacency_matrix(G)).todense()
)
pcae = cluster.AgglomerativeClustering(
    n_clusters=n_clusters, 
    affinity='cosine', 
    linkage='average'
).fit(pcae).labels_

print("Isomap Score:")
x = evalClusteringOnLabels(pcae, labels.cluster)

Isomap Score:
adj. MI score:   0.62
adj. RAND score: 0.53
F-M score:       0.55


In [17]:
pcae = manifold.LocallyLinearEmbedding(
    n_components=embed_size,
    n_neighbors=13,
)
pcae = pcae.fit_transform(
    _sparse_normalize_rows(nx.adjacency_matrix(G)).todense()
)
pcae = cluster.AgglomerativeClustering(
    n_clusters=n_clusters, 
    affinity='cosine', 
    linkage='average'
).fit(pcae).labels_

print("Isomap Score:")
evalClusteringOnLabels(pcae, labels.cluster)

Isomap Score:
adj. MI score:   0.59
adj. RAND score: 0.52
F-M score:       0.54


array([0.58612166, 0.51612726, 0.53898639])